## Problem Statement

### Business Context

The healthcare industry is rapidly evolving, with professionals facing increasing challenges in managing vast volumes of medical data while delivering accurate and timely diagnoses. The need for quick access to comprehensive, reliable, and up-to-date medical knowledge is critical for improving patient outcomes and ensuring informed decision-making in a fast-paced environment.

Healthcare professionals often encounter information overload, struggling to sift through extensive research and data to create accurate diagnoses and treatment plans. This challenge is amplified by the need for efficiency, particularly in emergencies, where time-sensitive decisions are vital. Furthermore, access to trusted, current medical information from renowned manuals and research papers is essential for maintaining high standards of care.

To address these challenges, healthcare centers can focus on integrating systems that streamline access to medical knowledge, provide tools to support quick decision-making, and enhance efficiency. Leveraging centralized knowledge platforms and ensuring healthcare providers have continuous access to reliable resources can significantly improve patient care and operational effectiveness.

**Common Questions to Answer**

**1. Diagnostic Assistance**: "What are the common symptoms and treatments for pulmonary embolism?"

**2. Drug Information**: "Can you provide the trade names of medications used for treating hypertension?"

**3. Treatment Plans**: "What are the first-line options and alternatives for managing rheumatoid arthritis?"

**4. Specialty Knowledge**: "What are the diagnostic steps for suspected endocrine disorders?"

**5. Critical Care Protocols**: "What is the protocol for managing sepsis in a critical care unit?"

### Objective

As an AI specialist, your task is to develop a RAG-based AI solution using renowned medical manuals to address healthcare challenges. The objective is to **understand** issues like information overload, **apply** AI techniques to streamline decision-making, **analyze** its impact on diagnostics and patient outcomes, **evaluate** its potential to standardize care practices, and **create** a functional prototype demonstrating its feasibility and effectiveness.

### Data Description

The **Merck Manuals** are medical references published by the American pharmaceutical company Merck & Co., that cover a wide range of medical topics, including disorders, tests, diagnoses, and drugs. The manuals have been published since 1899, when Merck & Co. was still a subsidiary of the German company Merck.

The manual is provided as a PDF with over 4,000 pages divided into 23 sections.

In [79]:
!jupyter nbconvert --to html

This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True]
-y
    Answer yes to any questions instead of prompting.
    Equivalent to: [--JupyterApp.answer_yes=True]
--execute
    Execute the notebook prior to export.
    Equivalent to: [--ExecutePr

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Installing and Importing Necessary Libraries and Dependencies

In [2]:
# Uninstall conflicting packages
!pip uninstall -y llma-cpp-python numpy pandas langchain langchain-core langchain-community sentence-transformers

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
Found existing installation: pandas 2.2.2
Uninstalling pandas-2.2.2:
  Successfully uninstalled pandas-2.2.2


In [3]:
# Install clean versions
!pip install numpy==1.26.4 pandas==2.2.2 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 80.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 110.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [4]:
# Installation for GPU llama-cpp-python
# uncomment and run the following code in case GPU is being used
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.85 --force-reinstall --no-cache-dir -q

# Installation for CPU llama-cpp-python
# uncomment and run the following code in case GPU is not being used
# !CMAKE_ARGS="-DLLAMA_CUBLAS=off" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.85 --force-reinstall --no-cache-dir -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 26.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 316.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 243.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 274.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 284.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.61.2 requires numpy<2.3,>=1.24, but you have numpy 2.3.1 which is incompatible.


In [42]:
# For installing the libraries & downloading models from HF Hub
!pip install huggingface_hub==0.23.2 pandas==1.5.3 tiktoken==0.6.0 pymupdf==1.25.1 langchain==0.1.17 langchain-core==0.1.51 langchain-community==0.0.36 chromadb==0.4.22 sentence-transformers==2.3.1 numpy==1.25.2 -q
!pip install posthog==3.0.1  # installing a compatible version

  Attempting uninstall: posthog
    Found existing installation: posthog 6.0.0
    Uninstalling posthog-6.0.0:
      Successfully uninstalled posthog-6.0.0


In [1]:
#Libraries for processing dataframes,text
import json,os
import tiktoken
import pandas as pd

#Libraries for Loading Data, Chunking, Embedding, and Vector Databases
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma

#Libraries for downloading and loading the llm
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

## Question Answering using LLM

#### Downloading and Loading the model

In [2]:
model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.2-GGUF"
model_basename = "mistral-7b-instruct-v0.2.Q6_K.gguf"

In [3]:
model_path = hf_hub_download(
    repo_id=model_name_or_path,
    filename=model_basename
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
# As Runtime is connected to GPU.
llm = Llama(
    model_path=model_path,
    n_ctx=4500,   # Context Window Size
    n_gpu_layers=38,
    n_batch=512
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


#### Response

In [5]:
def response(query,max_tokens=128,temperature=0,top_p=0.95,top_k=50):
    model_output = llm(
      prompt=query,
      max_tokens=max_tokens,
      temperature=temperature,
      top_p=top_p,
      top_k=top_k
    )

    return model_output['choices'][0]['text']

### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [6]:
user_input = "What is the protocol for managing sepsis in a critical care unit?"
response(user_input)

'\n\nSepsis is a life-threatening condition that can arise from an infection, and it requires prompt recognition and aggressive management in a critical care unit. The following are general steps for managing sepsis in a critical care unit:\n\n1. Early recognition: Recognize the signs and symptoms of sepsis early and initiate treatment as soon as possible. Sepsis can present with various clinical features, including fever or hypothermia, tachycardia or bradycardia, altered mental status, respiratory distress, and lactic acidosis.\n2. Source control'

### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [7]:
user_input = "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"
response(user_input)

Llama.generate: prefix-match hit


'\n\nAppendicitis is a medical condition characterized by inflammation of the appendix, a small tube-shaped organ located in the lower right side of the abdomen. The symptoms of appendicitis can vary from person to person, but some common signs include:\n\n1. Abdominal pain: The pain is typically located in the lower right quadrant of the abdomen and may start as a mild discomfort that gradually worsens over time. The pain may be constant or intermittent and may be aggravated by movement, deep breathing, or coughing.\n2. Loss of appetite'

### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [8]:
user_input = "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?"
response(user_input)

Llama.generate: prefix-match hit


"\n\nSudden patchy hair loss, also known as alopecia areata, is a common autoimmune disorder that affects the hair follicles. It can result in round or oval bald patches on the scalp, but it can also occur on other parts of the body such as the beard area, eyebrows, and eyelashes.\n\nThe exact cause of alopecia areata is not known, but it's believed to be related to a problem with the immune system. Some possible triggers for this condition include stress, genetics, viral infections, and certain medications."

### Query 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [9]:
user_input = "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"
response(user_input)

Llama.generate: prefix-match hit


"\n\nA person who has sustained a physical injury to brain tissue, also known as a traumatic brain injury (TBI), may require various treatments depending on the severity and location of the injury. Here are some common treatments recommended for TBIs:\n\n1. Emergency care: The first priority is to ensure the person's airway is clear, they are breathing, and their heart is beating normally. In severe cases, emergency surgery may be required to remove hematomas or other obstructions.\n2. Medications: Depending on the symptoms, medications may be prescribed to manage conditions such as"

### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [10]:
user_input = "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"
response(user_input)

Llama.generate: prefix-match hit


"\n\nFirst and foremost, if you suspect that someone has fractured their leg while hiking, it's essential to ensure their safety and prevent further injury. Here are some necessary precautions:\n\n1. Keep the person calm and still: Encourage them to remain as still as possible to minimize pain and prevent worsening the injury.\n2. Assess the situation: Check for any signs of shock, such as pale skin, rapid heartbeat, or low blood pressure. If you notice these symptoms, seek medical help immediately.\n3. Immobilize the leg: Use a splint,"

## Question Answering using LLM with Prompt Engineering

### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [11]:
system_prompt = "You are a highly knowledgeable and helpful assistant specialized in answering questions accurately and concisely. Always provide clear, correct, and context-aware answers. When relevant, explain reasoning briefly, cite sources if available, and ask clarifying questions if the user's input is ambiguous. If a question is outside your scope or lacks enough information, state that explicitly rather than guessing."
user_input = system_prompt+"\n"+ "What is the protocol for managing sepsis in a critical care unit?"
response(user_input)

Llama.generate: prefix-match hit


'\nSepsis is a life-threatening condition caused by a dysregulated response to infection. In a critical care unit, managing sepsis involves several steps:\n1. Early recognition and diagnosis: Identify sepsis suspects based on clinical suspicion, laboratory findings (e.g., leukocytosis or leukopenia, lactic acidosis), and vital sign abnormalities (e.g., hypotension, tachycardia).\n2. Immediate fluid resuscitation: Administer intravenous fluids to maintain adequate tissue perfusion and organ function.'

### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [12]:
user_input = system_prompt+"\n"+ "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"
response(user_input)

Llama.generate: prefix-match hit


'\nAppendicitis is a medical condition characterized by inflammation of the appendix, a small pouch located in the lower right side of the abdomen. Common symptoms include:\n1. Sudden and persistent pain in the lower right abdomen, often starting around the navel and moving to the right.\n2. Loss of appetite and feeling sick to your stomach (nausea).\n3. Fever, which may be low-grade at first but can rise as high as 103°F (39.4°C) or more.\n4. Vomiting, especially'

### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [13]:
user_input = system_prompt+"\n"+ "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?"
response(user_input)

Llama.generate: prefix-match hit


"\n\nSudden patchy hair loss, also known as alopecia areata, is an autoimmune condition that results in the immune system attacking hair follicles. The exact cause of alopecia areata is not known, but it's believed to be related to genetics and environmental factors.\n\nEffective treatments for addressing sudden patchy hair loss include:\n\n1. Corticosteroids: These medications can help reduce inflammation and suppress the immune system response that damages hair follicles. They can be applied topically or taken orally.\n2. Immunotherapy"

### Query 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [14]:
user_input = system_prompt+"\n"+ "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"
response(user_input)

Llama.generate: prefix-match hit


'\n\nFor a person with a brain injury, the specific treatment depends on the severity and location of the injury. Here are some common treatments:\n\n1. Medications: Doctors may prescribe medications to manage symptoms such as pain, seizures, or depression. Some drugs can help improve cognitive function, reduce inflammation, or protect neurons from further damage.\n\n2. Rehabilitation therapy: This includes physical therapy, occupational therapy, speech-language therapy, and cognitive rehabilitation. These therapies aim to help the person regain lost skills and compensate for impairments.\n\n'

### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [15]:
user_input = system_prompt+"\n"+ "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"
response(user_input)

Llama.generate: prefix-match hit


"\nA fractured leg is a serious injury that requires immediate medical attention. Here are some general precautions and treatment steps for a person with a leg fracture during a hiking trip:\n1. Do not move the person unless it's necessary to prevent further harm or ensure their safety. If the fracture is open (compound), try to keep the injured area clean and covered with a sterile dressing.\n2. Apply a splint or immobilize the leg using a sling, brace, or other supportive device to prevent movement and reduce pain.\n3. Provide comfort and reassurance to the"

## Data Preparation for RAG

In [16]:
#Libraries for processing dataframes,text
import json,os
import tiktoken
import pandas as pd

#Libraries for Loading Data, Chunking, Embedding, and Vector Databases
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma

### Loading the Data

In [17]:
medical_diagnosis_pdf_path = "/content/drive/MyDrive/AIML-UTAustin/Project 5_Medical Assistance/medical_diagnosis_manual.pdf"

In [18]:
pdf_loader = PyMuPDFLoader(medical_diagnosis_pdf_path)

In [19]:
manual = pdf_loader.load()

### Data Overview

#### Checking the first 5 pages

In [20]:
for i in range(5):
    print(f"Page Number : {i+1}",end="\n")
    print(manual[i].page_content,end="\n")

Page Number : 1
deeparajasabeson@gmail.com
AZS5UMLV8K
 for personal use by deeparajasabeson@
shing the contents in part or full is liable 

Page Number : 2
deeparajasabeson@gmail.com
AZS5UMLV8K
This file is meant for personal use by deeparajasabeson@gmail.com only.
Sharing or publishing the contents in part or full is liable for legal action.

Page Number : 3
Table of Contents
1
Front    ................................................................................................................................................................................................................
1
Cover    .......................................................................................................................................................................................................
2
Front Matter    ...........................................................................................................................................................................

#### Checking the number of pages

In [21]:
len(manual)

4114

### Data Chunking

In [22]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name='cl100k_base',
    chunk_size=512,
    chunk_overlap= 20
)

In [23]:
document_chunks = pdf_loader.load_and_split(text_splitter)
len(document_chunks)

8487

In [24]:
document_chunks[0].page_content

'deeparajasabeson@gmail.com\nAZS5UMLV8K\n for personal use by deeparajasabeson@\nshing the contents in part or full is liable'

In [25]:
document_chunks[1].page_content

'deeparajasabeson@gmail.com\nAZS5UMLV8K\nThis file is meant for personal use by deeparajasabeson@gmail.com only.\nSharing or publishing the contents in part or full is liable for legal action.'

In [26]:
document_chunks[2].page_content

'Table of Contents\n1\nFront    ................................................................................................................................................................................................................\n1\nCover    .......................................................................................................................................................................................................\n2\nFront Matter    ...........................................................................................................................................................................................\n53\n1 - Nutritional Disorders    ...............................................................................................................................................................\n53\nChapter 1. Nutrition: General Considerations    ...........................................................................................

In [27]:
document_chunks[4].page_content

'491\nChapter 44. Foot & Ankle Disorders    .....................................................................................................................................\n502\nChapter 45. Tumors of Bones & Joints    ...............................................................................................................................\n510\n5 - Ear, Nose, Throat & Dental Disorders    ..................................................................................................................\n510\nChapter 46. Approach to the Patient With Ear Problems    ...........................................................................................\n523\nChapter 47. Hearing Loss    .........................................................................................................................................................\n535\nChapter 48. Inner Ear Disorders    ...................................................................................................

In [28]:
document_chunks[5].page_content

'742\n7 - Dermatologic Disorders    ....................................................................................................................................................\n742\nChapter 71. Approach to the Dermatologic Patient    .......................................................................................................\n755\nChapter 72. Principles of Topical Dermatologic Therapy    ............................................................................................\n760\nChapter 73. Acne & Related Disorders    ...............................................................................................................................\n766\nChapter 74. Bullous Diseases    .................................................................................................................................................\n771\nChapter 75. Cornification Disorders    .............................................................................................

### Embedding

In [34]:
# embedding_model = SentenceTransformerEmbeddings(model_name='thenlper/gte-large')  # taking long time to create Vector DB
embedding_model = SentenceTransformerEmbeddings(model_name='all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [36]:
embedding_1 = embedding_model.embed_query(document_chunks[0].page_content)
embedding_2 = embedding_model.embed_query(document_chunks[1].page_content)

In [37]:
print("Dimension of the embedding vector ",len(embedding_1))
len(embedding_1)==len(embedding_2)

Dimension of the embedding vector  384


True

* The embedding model provides a fixed-length vector for any number of chunks.  
* This is necessary because we want to compare them for similarity.

### Vector Database

In [38]:
out_dir = 'medical_db'

if not os.path.exists(out_dir):
  os.makedirs(out_dir)

In [39]:
vectorstore = Chroma.from_documents(
    document_chunks,
    embedding_model,
    persist_directory=out_dir
)

In [40]:
vectorstore = Chroma(persist_directory=out_dir,embedding_function=embedding_model)

In [41]:
vectorstore.embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [47]:
vectorstore.similarity_search("Nutritional Disorders",k=3)

[Document(page_content='Overnutrition may contribute to chronic disorders, such as cancer, hypertension, obesity, diabetes\nmellitus, and coronary artery disease. Dietary restrictions are necessary in many hereditary metabolic\ndisorders (eg, galactosemia, phenylketonuria).\nEvaluation of Nutritional Status\nIndications for nutritional evaluation include undesirable body weight or body composition, suspicion of\nspecific deficiencies or toxicities\n[Table 1-5. Recommended Dietary Intake for 40-yr-Olds with Moderate Physical Activity*]\nof essential nutrients, and, in infants and children, insufficient growth or development. Nutritional status\nshould be evaluated routinely as part of the clinical examination for infants and children, the elderly,\npeople taking several drugs, people with psychiatric disorders, and people with systemic disorders that\nlast longer than several days.\nThe Merck Manual of Diagnosis & Therapy, 19th Edition\nChapter 1. Nutrition: General Considerations\n56\n

### Retriever

In [48]:
retriever = vectorstore.as_retriever(
    search_type='similarity',
    search_kwargs={'k': 2}
)

In [49]:
rel_docs = retriever.get_relevant_documents("What is Pyoderma Gangrenosum and what are its Symptoms and Signs, Diagnosis and Treatment ?")
rel_docs

/usr/local/lib/python3.11/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[Document(page_content='Symptoms and Signs\nPyoderma gangrenosum begins as an inflamed erythematous papule, pustule, or nodule. The lesion,\nwhich may resemble a furuncle or an arthropod bite at this stage, then ulcerates and expands rapidly,\ndeveloping a swollen necrotic base and a raised dusky to violaceous border. An undermined border is\ncommon, if not pathognomonic. Systemic symptoms such as fever, malaise, and arthralgias are common.\nThe ulcers coalesce to form larger ulcers, often with cribriform or sieve-like scarring.\nDiagnosis\nDiagnosis is clinical. Biopsies of lesions are not often diagnostic but may be supportive; 40% of biopsies\nfrom a leading edge show vasculitis with neutrophils and fibrin in superficial vessels.\nTreatment\n• Corticosteroids\n• Sometimes other anti-inflammatory drugs or immunosuppressants\n• Avoidance of surgical debridement\nPrednisone 60 to 80 mg po once/day is still the mainstay of treatment, although cyclosporine 3 mg/kg po\nonce/day is also qu

In [52]:
model_output = llm (
    prompt="what is Bromhidrosis",
    max_tokens=4500,
    temperature=10
)
print(model_output)

Llama.generate: prefix-match hit


{'id': 'cmpl-b8c160fd-b913-41ae-a6c0-7a8bda2dbdf3', 'object': 'text_completion', 'created': 1751248873, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Mistral-7B-Instruct-v0.2-GGUF/snapshots/3a6fbf4a41a1d52e415a4958cde6856d34b2db93/mistral-7b-instruct-v0.2.Q6_K.gguf', 'choices': [{'text': " ?\n\nBromhidrosis is a condition characterized by a persistent unpleasant body odor. Although it commonly involves the underarms and feet, any area of the body rich in apocrine glands can be affected (groin region or breast for females, beard regions or areas under excessive foreskin or armpit creases in men) where apocrine sweating takes place due to their ability to foster bacterial overgrowth and consequential breakdown products leading to obnoxious smelling chemicals like skatole and putresciene which cause a foul smell when mixed with the lipid base of apocrine sweat. While excessive perspiration called Hyperhidrosis may lead to increased opportunities for microbial growth in Bromhidr

The above response is somewhat generic and is solely based on the data the model was trained on, rather than the medical manual.

Let's now provide our own context and align the response with our needs.

### System and User Prompt Template

Prompts guide the model to generate accurate responses. Here, we define two parts:

    1. The system message describing the assistant's role.
    2. A user message template including context and the question.

In [53]:
qna_system_message = """
You are an assistant whose work is to review the report and provide the appropriate answers from the context.
User input will have the context required by you to answer user questions.
This context will begin with the token: ###Context.
The context contains references to specific portions of a document relevant to the user query.

User questions will begin with the token: ###Question.

Please answer only using the context provided in the input. Do not mention anything about the context in your final answer.

If the answer is not found in the context, respond "I don't know".
"""

In [54]:
qna_user_message_template = """
###Context
Here are some documents that are relevant to the question mentioned below.
{context}

###Question
{question}
"""

### Response Function

In [56]:
def generate_rag_response(user_input,k=3,max_tokens=128,temperature=0,top_p=0.95,top_k=50):
    global qna_system_message,qna_user_message_template

    # Retrieve relevant document chunks
    relevant_document_chunks = retriever.get_relevant_documents(query=user_input,k=k)
    context_list = [d.page_content for d in relevant_document_chunks]

    # Combine document chunks into a single context
    context_for_query = ". ".join(context_list)

    user_message = qna_user_message_template.replace('{context}', context_for_query)
    user_message = user_message.replace('{question}', user_input)

    prompt = qna_system_message + '\n' + user_message

    # Generate the response
    try:
        response = llm(
                  prompt=prompt,
                  max_tokens=max_tokens,
                  temperature=temperature,
                  top_p=top_p,
                  top_k=top_k
                  )

        # Extract and print the model's response
        response = response['choices'][0]['text'].strip()
    except Exception as e:
        response = f'Sorry, I encountered the following error: \n {e}'

    return response

## Question Answering using RAG

### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [57]:
user_input = "What is the protocol for managing sepsis in a critical care unit?"
generate_rag_response(user_input,top_k=20)

Llama.generate: prefix-match hit


"Based on the context provided, the protocol for managing sepsis in a critical care unit includes providing supplemental O2 by face mask, airway intubation with mechanical ventilation if necessary, inserting two large IV catheters into separate peripheral veins, and infusing 1 L of 0.9% saline over 15 minutes or using Ringer's lactate in major hemorrhage. The patient should be admitted to an ICU for monitoring, which includes ECG, systolic, diastolic, and mean BP, respiratory rate and depth, pulse o"

### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [58]:
user_input = "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"
generate_rag_response(user_input,top_k=20)

Llama.generate: prefix-match hit


"Based on the context provided, the common symptoms for appendicitis include epigastric or periumbilical pain followed by brief nausea, vomiting, anorexia, and after a few hours, the pain shifts to the right lower quadrant. Pain increases with cough and motion. Classic signs are right lower quadrant direct and rebound tenderness located at McBurney's point. Additional signs include pain felt in the right lower quadrant with palpation of the left lower quadrant (Rovsing sign), an increase in pain from passive extension of the right hip joint that stretches"

### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [59]:
user_input = "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?"
generate_rag_response(user_input,top_k=20)

Llama.generate: prefix-match hit


'Based on the context provided, the condition being described is Alopecia Areata, which is a type of nonscarring alopecia. The context mentions that it is an autoimmune disorder affecting genetically susceptible people exposed to unclear environmental triggers. There is no definitive cure for Alopecia Areata, but treatments can help promote regrowth and prevent further hair loss. Some of the treatments mentioned in the context include:\n\n1. Corticosteroids: These are anti-inflammatory drugs that can be applied topically or taken orally to reduce infl'

### Query 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [60]:
user_input = "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"
generate_rag_response(user_input,top_k=20)

Llama.generate: prefix-match hit


'Answer: The treatments recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function include preventive measures such as preventing systemic complications due to immobilization (eg, pneumonia, UTI, thromboembolic disease), providing good nutrition, preventing pressure ulcers, and physical therapy to prevent limb contractures. There is no specific treatment for the injury itself.'

### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [61]:
user_input = "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"
generate_rag_response(user_input,top_k=20)

Llama.generate: prefix-match hit


'Based on the context provided, the following are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip:\n1. The first step is to immobilize the injured leg to prevent further injury and decrease pain. This can be done by splinting with a nonrigid or noncircumferential device.\n2. Pain should be treated, typically with opioids.\n3. Definitive treatment often involves reduction, which requires analgesia or sedation. Closed reduction is maintained by casting, while open reduction is maintained by various surgical hardware'

### Fine-tuning

Query 1: What is the protocol for managing sepsis in a critical care unit?

In [62]:
user_input = "What is the protocol for managing sepsis in a critical care unit?"
generate_rag_response(user_input,temperature=0.5)

Llama.generate: prefix-match hit


"From the context, the protocol for managing sepsis in a critical care unit includes providing supplemental O2 by face mask, airway intubation with mechanical ventilation if necessary, insertion of two large IV catheters into separate peripheral veins or use of a central venous line or intraosseous needle, infusion of fluids such as 0.9% saline or Ringer's lactate, and monitoring of various physiologic parameters including ECG, systolic, diastolic, and mean BP, respiratory rate and depth, pulse oximetry"

Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [63]:
user_input = "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"
generate_rag_response(user_input,temperature=0.5)

Llama.generate: prefix-match hit


"Based on the context provided, the common symptoms for appendicitis include epigastric or periumbilical pain followed by brief nausea, vomiting, anorexia, and after a few hours, the pain shifts to the right lower quadrant. The pain increases with cough and motion. Classic signs are right lower quadrant direct and rebound tenderness located at McBurney's point. Other signs include pain felt in the right lower quadrant with palpation of the left lower quadrant (Rovsing sign), an increase in pain from passive extension of the right hip joint that stret"

Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [64]:
user_input = "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?"
generate_rag_response(user_input,temperature=0.5)

Llama.generate: prefix-match hit


"From the context, it appears that sudden patchy hair loss is a symptom of alopecia areata. The context also mentions that alopecia areata is thought to be an autoimmune disorder affecting genetically susceptible people exposed to unclear environmental triggers. No specific treatments or solutions for addressing this condition have been mentioned in the context, but it does suggest microscopic hair examination or scalp biopsy for definitive diagnosis. Therefore, I don't know if there are any effective treatments or solutions mentioned in the context for sudden patchy hair loss. However, based on the"

Query 4: What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [65]:
user_input = "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"
generate_rag_response(user_input,temperature=0.5)

Llama.generate: prefix-match hit


'Answer: The treatments recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function include preventive measures against systemic complications such as pneumonia, UTI, thromboembolic disease, providing good nutrition, preventing pressure ulcers, and in some cases, physical therapy to prevent limb contractures.'

Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [66]:
user_input = "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"
generate_rag_response(user_input,temperature=0.5)

Llama.generate: prefix-match hit


'Answer:\nAfter a leg fracture, the first step is to immobilize the injured area using a splint or sling to prevent further injury and decrease pain. The RICE method (rest, ice, compression, elevation) can also be used to minimize swelling and promote healing. Definitive treatment may involve reduction, which often requires analgesia or sedation. Closed reduction is maintained by casting, while open reduction is maintained by surgical hardware. Once the fracture has been stabilized, rehabilitation should begin as soon as possible to increase strength and prevent atrophy on the una'

## Output Evaluation

Let us now use the LLM-as-a-judge method to check the quality of the RAG system on two parameters - retrieval and generation.

We are using the same Mistral model for evaluation, so basically here the llm is rating itself on how well he has performed in the task.

In [67]:
groundedness_rater_system_message  = """
You are tasked with rating AI generated answers to questions posed by users.
You will be presented a question, context used by the AI system to generate the answer and an AI generated answer to the question.
In the input, the question will begin with ###Question, the context will begin with ###Context while the AI generated answer will begin with ###Answer.

Evaluation criteria:
The task is to judge the extent to which the metric is followed by the answer.
1 - The metric is not followed at all
2 - The metric is followed only to a limited extent
3 - The metric is followed to a good extent
4 - The metric is followed mostly
5 - The metric is followed completely

Metric:
The answer should be derived only from the information presented in the context

Instructions:
1. First write down the steps that are needed to evaluate the answer as per the metric.
2. Give a step-by-step explanation if the answer adheres to the metric considering the question and context as the input.
3. Next, evaluate the extent to which the metric is followed.
4. Use the previous information to rate the answer using the evaluaton criteria and assign a score.
"""

In [68]:
relevance_rater_system_message = """
You are tasked with rating AI generated answers to questions posed by users.
You will be presented a question, context used by the AI system to generate the answer and an AI generated answer to the question.
In the input, the question will begin with ###Question, the context will begin with ###Context while the AI generated answer will begin with ###Answer.

Evaluation criteria:
The task is to judge the extent to which the metric is followed by the answer.
1 - The metric is not followed at all
2 - The metric is followed only to a limited extent
3 - The metric is followed to a good extent
4 - The metric is followed mostly
5 - The metric is followed completely

Metric:
Relevance measures how well the answer addresses the main aspects of the question, based on the context.
Consider whether all and only the important aspects are contained in the answer when evaluating relevance.

Instructions:
1. First write down the steps that are needed to evaluate the context as per the metric.
2. Give a step-by-step explanation if the context adheres to the metric considering the question as the input.
3. Next, evaluate the extent to which the metric is followed.
4. Use the previous information to rate the context using the evaluaton criteria and assign a score.
"""

In [69]:
user_message_template = """
###Question
{question}

###Context
{context}

###Answer
{answer}
"""

# Defining the Evaluation Function

In [71]:
def generate_ground_relevance_response(user_input,k=3,max_tokens=128,temperature=0,top_p=0.95,top_k=50):
    global qna_system_message,qna_user_message_template
    # Retrieve relevant document chunks
    relevant_document_chunks = retriever.get_relevant_documents(query=user_input,k=3)
    context_list = [d.page_content for d in relevant_document_chunks]
    context_for_query = ". ".join(context_list)

    # Combine user_prompt and system_message to create the prompt
    prompt = f"""[INST]{qna_system_message}\n
                {'user'}: {qna_user_message_template.format(context=context_for_query, question=user_input)}
                [/INST]"""

    response = llm(
            prompt=prompt,
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            stop=['INST'],
            echo=False
            )

    answer =  response["choices"][0]["text"]

    # Combine user_prompt and system_message to create the prompt
    groundedness_prompt = f"""[INST]{groundedness_rater_system_message}\n
                {'user'}: {user_message_template.format(context=context_for_query, question=user_input, answer=answer)}
                [/INST]"""

    # Combine user_prompt and system_message to create the prompt
    relevance_prompt = f"""[INST]{relevance_rater_system_message}\n
                {'user'}: {user_message_template.format(context=context_for_query, question=user_input, answer=answer)}
                [/INST]"""

    response_1 = llm(
            prompt=groundedness_prompt,
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            stop=['INST'],
            echo=False
            )

    response_2 = llm(
            prompt=relevance_prompt,
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            stop=['INST'],
            echo=False
            )

    return response_1['choices'][0]['text'],response_2['choices'][0]['text']

Query 1: What is the protocol for managing sepsis in a critical care unit?

In [72]:
ground,rel = generate_ground_relevance_response(user_input="What is the protocol for managing sepsis in a critical care unit?",max_tokens=370)

print(ground,end="\n\n")
print(rel)

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


 Steps to evaluate the answer:
1. Identify the key information in the context related to managing sepsis in a critical care unit.
2. Determine if the AI generated answer includes only the information from the context.
3. If yes, rate the answer based on the evaluation criteria.

Explanation:
The context provides detailed information about the approach to managing critically ill patients in an ICU, including supportive care and patient monitoring. It also discusses various tests and treatments for different conditions, such as shock. The question asks specifically about the protocol for managing sepsis in a critical care unit.

The AI generated answer includes the following information:
1. Treatment begins simultaneously with evaluation.
2. Supplemental O2 by face mask is provided.
3. Airway intubation with mechanical ventilation is necessary if shock is severe or if ventilation is inadequate.
4. Two large IV catheters are inserted into separate peripheral veins.
5. Ringer's lactate is 

Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [73]:
ground,rel = generate_ground_relevance_response(user_input="What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?",max_tokens=370) #Complete the code to pass the query #2 along with parameters if needed

print(ground,end="\n\n")
print(rel)

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


 Steps to evaluate the answer:
1. Identify the common symptoms for appendicitis mentioned in the context.
2. Check if the AI generated answer includes all the identified common symptoms.
3. Determine if the AI generated answer mentions that appendicitis cannot be cured via medicine and that surgical removal is the treatment of choice.
4. Verify that the information in the AI generated answer is derived only from the context provided.

The AI generated answer adheres to the metric as it includes all the common symptoms mentioned in the context and correctly states that appendicitis cannot be cured via medicine and that surgical removal is the treatment of choice. Therefore, the metric is followed completely.

Rating: 5 (The metric is followed completely)

 Steps to evaluate the context as per the relevance metric:
1. Identify the main aspects of the question: common symptoms for appendicitis and whether it can be cured via medicine or if surgery is required.
2. Determine if the context 

Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [74]:
ground,rel = generate_ground_relevance_response(user_input="What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?",max_tokens=370)

print(ground,end="\n\n")
print(rel)

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


 Steps to evaluate the answer:
1. Identify the question and the key information in the context related to the question.
2. Determine if the AI generated answer is derived only from the information presented in the context.
3. Check if the possible causes mentioned in the answer are explicitly stated in the context.
4. Evaluate the extent to which the metric is followed.

Explanation:
The question asks about effective treatments or solutions for addressing sudden patchy hair loss and its possible causes. The context provides information on various types of alopecia, including androgenetic alopecia and alopecia areata. Androgenetic alopecia is mentioned as the most common cause of hair loss, but no specific treatments or solutions for this condition are provided in the context. However, the context does mention that microscopic hair examination or scalp biopsy may be required for definitive diagnosis of alopecia areata.

The AI generated answer correctly identifies alopecia areata as a p

Query 4: What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [75]:
ground,rel = generate_ground_relevance_response(user_input="What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?",max_tokens=370)

print(ground,end="\n\n")
print(rel)

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


 Steps to evaluate the answer:
1. Identify the key information related to treatments from both the question and context.
2. Compare the identified information with the AI generated answer to check if it adheres to the metric.
3. Evaluate the extent to which the metric is followed.

Explanation:
The question asks for the recommended treatments for a person with a brain injury, while the context provides information about supportive care that should be provided to prevent complications and maintain overall health of the patient. The AI generated answer correctly identifies preventing systemic complications, providing good nutrition, preventing pressure ulcers, and physical therapy as the recommended treatments. These treatments are directly derived from the context, indicating that the metric is followed completely.

Rating:
Based on the evaluation criteria, I would rate the answer a 5 for following the metric completely. The answer adheres to the given metric by deriving all the informa

Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [76]:
ground,rel = generate_ground_relevance_response(user_input="What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?",max_tokens=370)

print(ground,end="\n\n")
print(rel)

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


 Steps to evaluate the answer:
1. Identify the information in the context related to fractured legs and necessary precautions and treatments.
2. Check if the AI generated answer includes all the necessary precautions and treatments mentioned in the context.
3. Verify that the information in the AI generated answer is derived only from the context provided.

Explanation:
The AI generated answer adheres to the metric as it includes all the necessary precautions and treatments for a person who has fractured their leg during a hiking trip, which are derived directly from the context. The answer mentions immobilizing the injury, applying RICE, treating pain with opioids, and undergoing reduction (closed or open) for definitive treatment, all of which are mentioned in the context. Additionally, the answer provides information on seeing a healthcare practitioner and prosthetist for skin breakdown, starting rehabilitation as soon as possible, and avoiding certain activities during recovery, al

## Actionable Insights and Business Recommendations

*   Vector database creation time increases with the number of pages in the PDF document.
*   Retrieval parameter k is critical as the answer can be spread across multiple contexts.
*   chunk_overlap ensures coherence, especially when context spans across chunks.
*   max_tokens depends on query complexity; higher values yield detailed responses, while simple queries result in concise outputs despite large token limits due to prompt design and zero temperature.
*   Refine prompt design and temperature settings to control response length and creativity.
*   Continuously adjust RAG parameters based on specific use cases for optimal performance.
*   Prioritize groundedness and relevance in evaluations to ensure reliable and contextually accurate outputs.
*   Establish a feedback loop to fine-tune parameters, improving performance for diverse query types.

# 📊 Business Insights

**Faster Information Access:**

Speeds up how medical assistants find answers from manuals.

**Improved Information Retrieval Efficiency:**

The RAG application significantly reduces the time required to find accurate answers from complex medical manuals, allowing medical assistants to make faster decisions during clinical workflows.

**Better & Enhanced Decision Support:**

By combining real-time language generation with trusted medical documentation, the model provides contextual and up-to-date support and helps to non-physician staff making informed decisions and in handling routine and emergency scenarios.

**Knowledge Democratization:**

Medical assistants often have limited access to high-level knowledge. This tool helps bridge that gap by providing simplified, accessible answers grounded in clinical documentation.

**Reduced Training Time:**

Reduces onboarding time for new medical assistants by providing consistent, easy-to-understand guidance to help them benefit from quick access to consistent information, potentially lowering onboarding time and reducing errors caused by misinterpretation or outdated resources.

**Scalable Customer Support Alternative:**

In settings like telemedicine or outpatient clinics, the model can act as a first-level query resolver, handling frequent or routine questions, thereby reducing the load on physicians or senior staff.

**Improved Workflow:**

Eases pressure on doctors by handling common or routine questions.

# ✅ Recommendations

**Integrate with Real-Time Data Sources:**

To further improve relevance, connect the application with real-time updates from clinical databases or hospital knowledge bases.

**Ensure Medical Accuracy via Human-in-the-Loop:**

In production, consider adding a validation layer involving a licensed practitioner to verify AI-generated answers for compliance and accuracy.

**Enhance Domain-Specific Training:**

Fine-tune the model on local hospital documentation or specific specialties (e.g., cardiology, dermatology) to make responses more relevant.

**Incorporate & Enable User Feedback Mechanism:**

Allow users (medical assistants) to rate or flag the answers for improvement. This feedback loop will help improve the system over time.

**Expand Language Support:**

Consider adding multilingual capabilities to serve diverse staff or patient populations, especially in multilingual healthcare environments.

**Privacy and Compliance Considerations:**

Ensure that all retrieved and generated content complies with HIPAA or local data privacy and medical regulations, especially if the system is extended to patient-specific queries.

<font size=6 color='blue'>Power Ahead</font>
___